In [1]:
import os
import sys 

path_folder = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/'
omitir = ''


def search_subfolders(path:str):
    '''Funcion para agregar rutas al path de ejecucion'''
    folder = []
    for root, dirs, _ in os.walk(path, topdown=False):
        for name in dirs:
            if name == omitir:
                print(f"[INFO] carpeta omitida: {name}")
            else:
                folder.append(os.path.join(root, name))
    return folder


for i in search_subfolders(path_folder):
    sys.path.insert(0, i)


In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import json
from factory_data import get_data, SQLDataSourceFactory
from factory_models import ModelContext, Modelos
from args_data_model import (
    ModelRNN,
    ModelBlockRNN,
    ModelExponentialSmoothing,
    ModelTCNModel,
    ModelFFT,
    ModelTransformerModel,
    ModelNBEATSModel,
    ModelDLinearModel,
    ModelNlinearModel,
    ModelTFTModel
)

attempted relative import with no known parent package


In [3]:
CONFIG_FILE = "/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/parameter/data_params_run.json"

with open(CONFIG_FILE, 'r', encoding='utf-8') as file:
    parameters = json.load(file)

print("Probando el estacion de datos de sql")
data_postgrest = get_data(SQLDataSourceFactory(**parameters))

Probando el estacion de datos de sql


In [4]:
with open(parameters['names_table_columns'] , 'r', encoding='utf-8') as file:
    names = json.load(file)

names['columns'] = list(names['columns'].values())

In [5]:
data_for_process = pd.DataFrame(data_postgrest,columns=names['columns'])

In [6]:
data_for_process.head()

,Fecha Creación,Material (Cod),Peso neto (TON),Solicitate (Cod),Cant Pedido UMV,Valor Neto
0,2022-01-01,10011308.000,10.000,0004155696,200.0,"3,844.5"
1,2022-01-02,10011308.000,10.000,0004158888,200.0,"3,939.1"
2,2022-01-03,10059053.000,34.190,0004151082,34.2,"11,863.9"
3,2022-01-03,10059053.000,33.330,0004139983,33.3,"11,346.9"
4,2022-01-03,10059053.000,33.610,0004150188,33.6,"11,662.7"


In [7]:
#Reempplazo de los tipo de datos por lo cuales 
#se va a cambiar el dataframe
new_types =[np.datetime64,int,int,'object',int,int]

#Estrategias para imputar los datos faltantes de NA 
strategy = {
    int:np.mean,
    float:np.mean,
    object:stats.mode
}

#metodo para transformar los tipo de datos
replace = {
    int:lambda x: int(float(x.replace(',',''))),
    float:lambda x: float(x.replace(',',''))
}

In [8]:
# from features_fix_data import ColumnsNameHandler,PrepareData

# HandleData = PrepareData(data_for_process)

# dict_update,trans_col = HandleData.get_dtypes_columns_update(new_types)

# HandleData.apply_transformations(trans_col,replace)

# HandleData.fill_na_columns(strategy)

# HandleData.update_dtypes(trans_col)

In [9]:
from factory_prepare_data import DataCleaner,MeanImputation

In [20]:
imputation = MeanImputation(
    replace_dtypes=new_types,
    strategy_imputation=strategy,
    preprocess_function=replace
)

cleaner = DataCleaner(imputation)
data_for_process = cleaner.clean(data_for_process)

In [26]:
CLIENTE = 4151082

In [27]:
COL_FEATURE ='Solicitate (Cod)'

In [29]:
data_for_process.dataframe[data_for_process.dataframe[COL_FEATURE] in str(CLIENTE)]

TypeError: 'in <string>' requires string as left operand, not Series

In [39]:
str(CLIENTE)  in data_for_process.dataframe['Solicitate (Cod)'][2]

True